In [151]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import col, max as max_
from pyspark.sql.window import Window
from pyspark.sql.functions import lag
spark = SparkSession.builder.appName('Monthly Percentage Difference').getOrCreate()

ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

In [ ]:
df_input = spark.read.options(header='True', InferSchema='True').csv('sf_transactions.csv')

In [ ]:
df_input.show()

In [ ]:
df_input.createOrReplaceTempView("transactions")
df_qty= spark.sql("select id,period,value,rn from "+ " (select  id,substring(created_at,1,7) as period,value, row_number() OVER (PARTITION BY id,substring(created_at,1,7) ORDER BY substring(created_at,1,7) ASC) as rn " +" FROM transactions) tmp")


In [ ]:
df_period_qty = df_qty.groupBy('period').sum('value').orderBy('period').select(col("period"),col("sum(value)").alias("total"))

In [ ]:
df_output= df_period_qty.withColumn("revenue_diff_pct", df_period_qty.total - lag(df_period_qty.total, 1).over(Window.partitionBy("period").orderBy("period")))

In [149]:
my_window = Window.partitionBy().orderBy("period")
df_qty = df_period_qty.withColumn("prev_value", F.lag(df_period_qty.total).over(my_window))

ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

In [150]:
df_qty.show()

AttributeError: 'NoneType' object has no attribute 'show'

In [ ]:
my_window = Window.partitionBy().orderBy("period")
df_qty = df_period_qty.withColumn("prev_value", F.lag(df_period_qty.total).over(my_window))

In [ ]:
df_output = df_qty.withColumn("revenue_diff_pct", ((df_qty['total'] -df_qty['prev_value']) / (df_qty['prev_value']))*100)

In [ ]:
df_output.selectExpr('period','revenue_diff_pct').show()